# Black Body Radiation Color Map

This iPython notebook contains the script required to derive a nice color map based on the colors of black body radiation. Although the colors are inspired by the wavelengths of light from black body radiation, the actual colors used are designed to be perceptually uniform. Colors of the desired brightness and hue are chosen, and then the colors are adjusted such that the luminance is perceptually linear (according to the CIELAB color space).

This code relies on the [python-colormath](http://python-colormath.readthedocs.org/en/latest/index.html) module. See [its documentation](http://python-colormath.readthedocs.org/en/latest/index.html) for information such as installation instructions. (It can be installed with either pip or macports.)

In [1]:
from colormath.color_objects import *
from colormath.color_conversions import convert_color

Mostly because it's habit, I am also using [pandas](http://pandas.pydata.org/) dataframes to organize the data. (Pandas can be installed with macports.)

In [2]:
import pandas
import numpy

We will also be using [toyplot](https://toyplot.readthedocs.org) for making visuals (version 0.10.0 or later required). See its documentation for installation instructions.

In [3]:
import toyplot
import toyplot.svg

This color map goes from black to red to orange to yellow to white. Create color objects for these control points.

In [4]:
color_rgb = [(0,  0,  0  ),    # black
             (178,34, 34 ),    # red
             (227,105,5  ),    # orange
             (230,230,53 ),    # yellow
             (255,255,255)]    # white
data = pandas.DataFrame({'RGB': color_rgb})
data

,RGB
0,"(0, 0, 0)"
1,"(178, 34, 34)"
2,"(227, 105, 5)"
3,"(230, 230, 53)"
4,"(255, 255, 255)"


In [5]:
data['rgb_values'] = data['RGB'].apply(lambda rgb: sRGBColor(rgb[0], rgb[1], rgb[2],
                                                             is_upscaled=True))

Convert the RGB values to LAB values and get the luminance out of each of them to define the scalar (on a scale from 0 to 1) of each color.

In [6]:
data['lab_values'] = data['rgb_values'].apply(lambda rgb: convert_color(rgb, LabColor))
data['scalar'] = data['lab_values'].apply(lambda lab: lab.lab_l/100.0)
data

,RGB,rgb_values,lab_values,scalar
0,"(0, 0, 0)",sRGBColor (rgb_r:0.0000 rgb_g:0.0000 rgb_b:0.0...,LabColor (lab_l:0.0000 lab_a:0.0000 lab_b:0.0000),0.000000
1,"(178, 34, 34)",sRGBColor (rgb_r:0.6980 rgb_g:0.1333 rgb_b:0.1...,LabColor (lab_l:39.1167 lab_a:55.9150 lab_b:37...,0.391167
2,"(227, 105, 5)",sRGBColor (rgb_r:0.8902 rgb_g:0.4118 rgb_b:0.0...,LabColor (lab_l:58.4605 lab_a:43.3378 lab_b:65...,0.584605
3,"(230, 230, 53)",sRGBColor (rgb_r:0.9020 rgb_g:0.9020 rgb_b:0.2...,LabColor (lab_l:88.7685 lab_a:-18.9553 lab_b:7...,0.887685
4,"(255, 255, 255)",sRGBColor (rgb_r:1.0000 rgb_g:1.0000 rgb_b:1.0...,LabColor (lab_l:100.0000 lab_a:-0.0005 lab_b:-...,1.000000


Make a summary table of just the control points.

In [7]:
control_points = pandas.DataFrame(data, columns=['scalar', 'RGB'])
control_points

,scalar,RGB
0,0.000000,"(0, 0, 0)"
1,0.391167,"(178, 34, 34)"
2,0.584605,"(227, 105, 5)"
3,0.887685,"(230, 230, 53)"
4,1.000000,"(255, 255, 255)"


Make a function that will take a scalar value (in the range of 0 and 1) and return the appropriate RGB triple.

In [8]:
def color_lookup_upscaled(x):
    if x < 0:
        return (0, 0, 0)
    for index in range(0, data.index.size-1):
        low_scalar = data['scalar'][index]
        high_scalar = data['scalar'][index+1]
        if (x > high_scalar):
            continue
        low_lab = data['lab_values'][index]
        high_lab = data['lab_values'][index+1]
        interp = (x-low_scalar)/(high_scalar-low_scalar)
        mid_lab = LabColor(interp*(high_lab.lab_l-low_lab.lab_l) + low_lab.lab_l,
                           interp*(high_lab.lab_a-low_lab.lab_a) + low_lab.lab_a,
                           interp*(high_lab.lab_b-low_lab.lab_b) + low_lab.lab_b,
                           observer=low_lab.observer,
                           illuminant=low_lab.illuminant)
        return convert_color(mid_lab, sRGBColor).get_upscaled_value_tuple()
    return (255, 255, 255)

def color_lookup(x):
    if x < 0:
        return (0.0, 0.0, 0.0)
    for index in range(0, data.index.size-1):
        low_scalar = data['scalar'][index]
        high_scalar = data['scalar'][index+1]
        if (x > high_scalar):
            continue
        low_lab = data['lab_values'][index]
        high_lab = data['lab_values'][index+1]
        interp = (x-low_scalar)/(high_scalar-low_scalar)
        mid_lab = LabColor(interp*(high_lab.lab_l-low_lab.lab_l) + low_lab.lab_l,
                           interp*(high_lab.lab_a-low_lab.lab_a) + low_lab.lab_a,
                           interp*(high_lab.lab_b-low_lab.lab_b) + low_lab.lab_b,
                           observer=low_lab.observer,
                           illuminant=low_lab.illuminant)
        return convert_color(mid_lab, sRGBColor).get_value_tuple()
    return (1.0, 1.0, 1.0)

Make a long table of colors. This is a very high resolution table of colors that can be easily trimmed down with regular sampling.

In [9]:
colors_long = pandas.DataFrame({'scalar': numpy.linspace(0.0, 1.0, num=1024)})
colors_long['RGB'] = colors_long['scalar'].apply(color_lookup_upscaled)
colors_long['sRGB'] = colors_long['scalar'].apply(color_lookup)

The colors are all stored as tuples in a single column. This is convenient for some operations, but not others. Thus, also create separate columns for the three RGB components.

In [10]:
def unzip_rgb_triple(dataframe, column='RGB'):
    '''Given a dataframe and the name of a column holding an RGB triplet,
    this function creates new separate columns for the R, G, and B values
    with the same name as the original with '_r', '_g', and '_b' appended.'''
    # Creates a data frame with separate columns for the triples in the given column
    unzipped_rgb = pandas.DataFrame(dataframe[column].values.tolist(),
                                    columns=['r', 'g', 'b'])
    # Add the columns to the original data frame
    dataframe[column + '_r'] = unzipped_rgb['r']
    dataframe[column + '_g'] = unzipped_rgb['g']
    dataframe[column + '_b'] = unzipped_rgb['b']

unzip_rgb_triple(control_points, 'RGB')
unzip_rgb_triple(colors_long, 'RGB')
unzip_rgb_triple(colors_long, 'sRGB')

Check to make sure that all the colors are actually valid. The answer to this sum should be 0 if all the values are valid.

In [11]:
invalid = ((colors_long['sRGB_r'] < 0) | (colors_long['sRGB_r'] > 1) |
           (colors_long['sRGB_g'] < 0) | (colors_long['sRGB_g'] > 1) |
           (colors_long['sRGB_b'] < 0) | (colors_long['sRGB_b'] > 1))
num_bad_values = invalid.sum()
if num_bad_values > 0:
    raise ValueError('Found %d invalid colors!!!!' % num_bad_values)

In [12]:
colors_long[invalid]

,scalar,RGB,sRGB,RGB_r,RGB_g,RGB_b,sRGB_r,sRGB_g,sRGB_b


Plot out the color map.

In [13]:
colors_palette = toyplot.color.Palette(colors=colors_long['sRGB'].values)
colors_map = toyplot.color.LinearMap(palette=colors_palette,
                                     domain_min=0, domain_max=1)

In [14]:
canvas = toyplot.Canvas(width=130, height=300)
numberline = canvas.numberline(x1=16, x2=16, y1=-7, y2=7)
numberline.padding = 5
numberline.axis.spine.show = False
numberline.colormap(colors_map,
                    width=30,
                    style={'stroke':'lightgrey'})

control_point_labels = \
    control_points.apply(lambda row: '%1.2f, %s' % (row['scalar'],
                                                  str(row['RGB'])),
                       axis=1)
numberline.axis.ticks.locator = \
    toyplot.locator.Explicit(locations=control_points['scalar'],
                             labels=control_point_labels)
numberline.axis.ticks.labels.angle = -90
numberline.axis.ticks.labels.style = {'text-anchor':'start',
                                      'baseline-shift':'0%',
                                      '-toyplot-anchor-shift':'15px'}

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="130.0px" height="300.0px" viewBox="0 0 130.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t001fa241c1a4498685177a8b1a754f1c"> 0.00, (0, 0, 0) 0.39, (178, 34, 34) 0.58, (227, 105, 5) 0.89, (230, 230, 53) 1.00, (255, 255, 255)

In [15]:
toyplot.svg.render(canvas, 'black-body.svg')

Create a color preset file for ParaView. Since ParaView 4.4, JSON files are supported, which makes it easy to export.

In [16]:
RGBPoints = []
for index in range(0, data.index.size):
    RGBPoints.append(data['scalar'][index])
    RGBPoints.extend(data['rgb_values'][index].get_value_tuple())
    
#RGBPoints

In [17]:
import json

file_descriptor = open('black-body-paraview-colors.json', 'w')
json.dump([{'ColorSpace':'Lab',
            'Name':'Black Body',
            'NanColor':[0.0,0.5,1.0],
            'RGBPoints':RGBPoints}],
          file_descriptor,
          indent=2)
file_descriptor.close()

Create several csv files containing color tables for this color map. We will create color tables of many different sizes from 8 rows to 1024. We also write out one set of csv files for "upscaled" color bytes (values 0-255) and another for floating point numbers (0-1).

In [18]:
for num_bits in range(3, 11):
    table_length = 2 ** num_bits
    color_table = pandas.DataFrame({'scalar': numpy.linspace(0.0, 1.0, num=table_length)})
    color_table['RGB'] = color_table['scalar'].apply(color_lookup_upscaled)
    unzip_rgb_triple(color_table, 'RGB')
    color_table.to_csv('black-body-table-byte-{:04}.csv'.format(table_length),
                       index=False,
                       columns=['scalar', 'RGB_r', 'RGB_g', 'RGB_b'])
    color_table['sRGB'] = color_table['scalar'].apply(color_lookup)
    unzip_rgb_triple(color_table, 'sRGB')
    color_table.to_csv('black-body-table-float-{:04}.csv'.format(table_length),
                       index=False,
                       columns=['scalar', 'sRGB_r', 'sRGB_g', 'sRGB_b'],
                       header=['scalar', 'RGB_r', 'RGB_g', 'RGB_b'])